In [1]:
import os
import libsbml
import logging
import sys
import pandas as pd
import numpy as np

In [2]:
logging.basicConfig(level=logging.INFO)
logging.info('Using libSBML: %s' % libsbml.getLibSBMLDottedVersion())

INFO:root:Using libSBML: 5.18.1


In [3]:
def read_excel_file(xls_file):
    df_metabs = pd.read_excel(xls_file, sheet_name='Metabolite List')
    df_reactions = pd.read_excel(xls_file, sheet_name='Reaction List')
    df_genes = pd.read_excel(xls_file, sheet_name='Gene List')
    logging.info('# metabolites: {0}'.format(df_metabs.shape[0]))
    logging.info('# reactions: {0}'.format(df_reactions.shape[0]))
    logging.info('# genes: {0}'.format(df_genes.shape[0]))
    return df_metabs, df_reactions, df_genes

In [4]:
def mangle_species_abbrev(abbrev):
    # type: (str) -> str
    """
    It seems in the SBML file the ids are mangled abbreviations, so for example
    we have a species with Id: 'M_10fthf__91__c__93__' and we have an
    abbreviation '10fthf[c]' so it would seem we just have to replace that
    and prefix it with 'M_'

    :param abbrev: the abbreviation from the metab table
    :return: hopefully an SBML id
    """
    sbml_id = abbrev.replace('[', '__91__')
    sbml_id = sbml_id.replace(']', '__93__')
    return 'M_' + sbml_id

In [5]:
def add_resource_to_term(term, current, column_header, url_template):
    try:
        if column_header not in current:
            return
        entry = current[column_header]
        if pd.isnull(entry) or entry == 0:
            return
        url = url_template.format(entry)
        term.addResource(url)
    except:
        logging.warning(u'invalid resource for {0} with entry {1} ... skipping'.format(unicode(column_header), unicode(entry)))
    pass


In [6]:
def add_metadata_to_species(species, current):
    # type: (libsbml.Species, pd.Series) -> None

    # first we have to figure out, whether the species has already metadata
    # otherwise we should create the metadata element first (skipped for now
    # since the file seems to contain metadata already)

    if species.getNumCVTerms() == 0:
        # here we would create it
        logging.warning('species {0} has no cvterms, skipped'.format(species.getId()))
        return

    # if so we just append it
    term = species.getCVTerm(0)
    if term.getBiologicalQualifierType() != libsbml.BQB_IS:
        logging.warning('term is of unexpected type, skipped')
        return

    # term is bqbiol:is, and so we can just add the resource if it is not present yet
    resources = []
    for i in range(term.getNumResources()):
        resources.append(term.getResourceURI(i))

    # NOTE: remember to actually check whether they are being used ...

    # add BioCyc ... should look like https://identifiers.org/biocyc:ECOLI:CYT-D-UBIOX-CPLX
    add_resource_to_term(term, current, 'BioCyc',
                         'https://identifiers.org/biocyc:{0}')

    # add bigg.metabolite .. looks like: 	https://identifiers.org/bigg.metabolite:12dgr161
    add_resource_to_term(term, current, 'bigg.metabolite',
                         'https://identifiers.org/bigg.metabolite:{0}')
    # add metanetx.chemical ... looks like 	https://identifiers.org/metanetx.chemical:MNXM1723
    add_resource_to_term(term, current, 'metanetx.chemical',
                         'https://identifiers.org/metanetx.chemical:{0}')
    # add reactome ... looks like: 	https://identifiers.org/reactome:R-HSA-201451
    add_resource_to_term(term, current, 'reactome',
                         'https://identifiers.org/reactome:{0}')
    # add SEED ID  ... looks like: 	https://identifiers.org/seed.compound:cpd15380
    add_resource_to_term(term, current, 'SEED ID',
                         'https://identifiers.org/seed.compound:{0}')
    # add HMDB database ... looks like: 	https://identifiers.org/hmdb:HMDB00001
    add_resource_to_term(term, current, 'HMDB database',
                         'https://identifiers.org/hmdb:{0}')
    # add SMILES ... TODO: figure out how it would look, does it get encoded as uri?
    # add InChI Code ... looks like https://identifiers.org/inchi:InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3
    add_resource_to_term(term, current, 'InChI Code',
                         'https://identifiers.org/inchi:{0}')
    # add InChiKey ... looks like 	https://identifiers.org/inchikey:RYYVLZVUVIJVGH-UHFFFAOYSA-N
    add_resource_to_term(term, current, 'InChiKey',
                         'https://identifiers.org/inchikey:{0}')
    # add ChEBI
    add_resource_to_term(term, current,'ChEBI',
                         'https://identifiers.org/CHEBI:{0}')
    add_resource_to_term(term, current, 'SBO',
                         'https://identifiers.org/SBO:{0}')

    
    pass

In [7]:
def add_metab_info(mod, metabs):
    # type: (libsbml.Model, pd.DataFrame) -> None
    #"""
    #this function takes a libsbml.Model, and a data frame with metab information
    #and adds them to the model.

    #:param mod: the SBML model
    #:param metabs: the metabolite dataframe
    #:return: None
    #"""

    # the plan is to go through each line, if we have a species according to
    # the row, we add the information, otherwise we skip to the next one
    for i in range(metabs.shape[0]):
        current = metabs.iloc[i]
        abbrev = current['Abbreviation']
        id = mangle_species_abbrev(abbrev)
        species = mod.getSpecies(id)
        if species is None:
            logging.info('no species for abbreviation %s' % abbrev)
            continue

        logging.info('adding metadata to {0}'.format(abbrev))
        add_metadata_to_species(species, current)

    pass

In [8]:
def add_metadata_to_reaction(reaction, current):
    # type: (libsbml.Reaction, pd.Series) -> None

    # first we have to figure out, whether the species has already metadata
    # otherwise we should create the metadata element first (skipped for now
    # since the file seems to contain metadata already)

    if reaction.getNumCVTerms() == 0:
        # here we would create it
        # logging.warning('reactions {0} has no cvterms, skipped'.format(reaction.getId()))
        cv = libsbml.CVTerm()
        cv.setQualifierType(libsbml.BIOLOGICAL_QUALIFIER)
        cv.setBiologicalQualifierType(libsbml.BQB_IS)
        reaction.addCVTerm(cv)
        logging.info('created term for %s' % reaction.getId())
        return

    # if so we just append it
    term = reaction.getCVTerm(0)
    # BQB_IS is a biomodels qualifier see: https://co.mbine.org/standards/qualifiers
    # in particular BQB_IS represents an 'is' qualifier for the biological aspects
    # which i found most useful ... also some reactions in your model still have one
    if term.getBiologicalQualifierType() != libsbml.BQB_IS:
        logging.warning('term is of unexpected type, skipped')
        return

    # term is bqbiol:is, and so we can just add the resource if it is not present yet
    resources = []
    for i in range(term.getNumResources()):
        resources.append(term.getResourceURI(i))

    # NOTE: remember to actually check whether they are being used ...

    add_resource_to_term(term, current, 'rhea',
                         'https://identifiers.org/rhea:{0}')
    add_resource_to_term(term, current, 'KEGG ID',
                         'https://identifiers.org/kegg.compound:{0}')
    add_resource_to_term(term, current, 'seed.reaction',
                         'https://identifiers.org/seed.reaction:{0}')
    add_resource_to_term(term, current, 'metanetx.reaction',
                         'https://identifiers.org/metanetx.reaction:{0}')
    add_resource_to_term(term, current, 'bigg.reaction',
                          'https://identifiers.org/bigg.reaction:{0}')
    add_resource_to_term(term, current, 'reactome',
                          'https://identifiers.org/reactome:{0}')
    add_resource_to_term(term, current, 'BioCyc',
                          'https://identifiers.org/biocyc:{0}')
    add_resource_to_term(term, current, 'SBO',
                          'https://identifiers.org/SBO:{0}')

    pass


In [9]:
def add_reaction_info(mod, reactions):
    # type: (libsbml.Model, pd.DataFrame) -> None
    """
    this function takes a libsbml.Model, and a data frame with reaction information
    and adds them to the model.

    :param mod: the SBML model
    :param reactions: the reaction dataframe
    :return: None
    """

    # the plan is to go through each line, if we have a species according to
    # the row, we add the information, otherwise we skip to the next one
    for i in range(reactions.shape[0]):
        current = reactions.iloc[i]
        # the reactions in cobra always start with 'R_' so we need to add that
        abbrev = current['Abbreviation']
        id = 'R_' + abbrev

        reaction = mod.getReaction(id)
        if reaction is None:
            logging.info(
                'in reaction no reaction for abbreviation %s' % abbrev)
            continue

        logging.info('adding metadata to {0}'.format(abbrev))
        add_metadata_to_reaction(reaction, current)

    pass

In [10]:
!cd

In [10]:
num_args = len(sys.argv)
sbml_file = 'iOpol_200814.xml'
xlsx_file = 'iOpol_200814.xlsx'

metabs, reactions, genes = read_excel_file(xlsx_file)

doc = libsbml.readSBMLFromFile(sbml_file)
assert (isinstance(doc, libsbml.SBMLDocument))

if doc.getNumErrors(libsbml.LIBSBML_SEV_ERROR) > 0:
    doc.printErrors()
    sys.exit(1)

mod = doc.getModel()

add_metab_info(mod, metabs)
add_reaction_info(mod, reactions)

libsbml.writeSBMLToFile(doc, 'iOpol_200820.xml')

INFO:root:# metabolites: 1639
INFO:root:# reactions: 2263
INFO:root:# genes: 909
INFO:root:adding metadata to oligoP[v]
INFO:root:adding metadata to polyP[v]
INFO:root:adding metadata to h2o[v]
INFO:root:adding metadata to h2o[c]
INFO:root:adding metadata to ppi[c]
INFO:root:adding metadata to h[c]
INFO:root:adding metadata to pi[c]
INFO:root:adding metadata to h2o[m]
INFO:root:adding metadata to ppi[m]
INFO:root:adding metadata to h[m]
INFO:root:adding metadata to pi[m]
INFO:root:adding metadata to allphn[c]
INFO:root:adding metadata to co2[c]
INFO:root:adding metadata to nh4[c]
INFO:root:adding metadata to h2o2[x]
INFO:root:adding metadata to o2[x]
INFO:root:adding metadata to h2o[x]
INFO:root:adding metadata to tre[c]
INFO:root:adding metadata to glc_D[c]
INFO:root:adding metadata to h2o[e]
INFO:root:adding metadata to tre[e]
INFO:root:adding metadata to glc_D[e]
INFO:root:adding metadata to tre[v]
INFO:root:adding metadata to glc_D[v]
INFO:root:adding metadata to thmpp[m]
INFO:root

INFO:root:adding metadata to ttc2coa[x]
INFO:root:adding metadata to hexccoa[x]
INFO:root:adding metadata to hxc2coa[x]
INFO:root:adding metadata to btcoa[x]
INFO:root:adding metadata to b2coa[x]
INFO:root:adding metadata to hxcoa[x]
INFO:root:adding metadata to hx2coa[x]
INFO:root:adding metadata to occoa[x]
INFO:root:adding metadata to oc2coa[x]
INFO:root:adding metadata to 3ohodcoa[x]
INFO:root:adding metadata to 3kc20coa[x]
INFO:root:adding metadata to 3kc22coa[x]
INFO:root:adding metadata to 3kc24coa[x]
INFO:root:adding metadata to 3ohxccoa[x]
INFO:root:adding metadata to succ[m]
INFO:root:adding metadata to succoa[m]
INFO:root:adding metadata to fad[m]
INFO:root:adding metadata to fum[m]
INFO:root:adding metadata to fadh2[m]
INFO:root:adding metadata to micit[m]
INFO:root:adding metadata to acgam1p[c]
INFO:root:adding metadata to uacgam[c]
INFO:root:adding metadata to for[c]
INFO:root:adding metadata to 25dhpp[c]
INFO:root:adding metadata to faptp[c]
INFO:root:adding metadata to 

INFO:root:adding metadata to 4mop[c]
INFO:root:adding metadata to leu_L[m]
INFO:root:adding metadata to 4mop[m]
INFO:root:adding metadata to ggl[c]
INFO:root:adding metadata to gal[c]
INFO:root:adding metadata to 5dpmev[c]
INFO:root:adding metadata to ipdp[c]
INFO:root:adding metadata to dmpp[c]
INFO:root:adding metadata to ins[c]
INFO:root:adding metadata to fprica[c]
INFO:root:adding metadata to xmp[c]
INFO:root:adding metadata to hxan[c]
INFO:root:adding metadata to dadp[c]
INFO:root:adding metadata to datp[c]
INFO:root:adding metadata to his_L[c]
INFO:root:adding metadata to NPmehis[c]
INFO:root:adding metadata to hstdal[c]
INFO:root:adding metadata to urcan[c]
INFO:root:adding metadata to 3ohcoa[x]
INFO:root:adding metadata to inost[c]
INFO:root:adding metadata to mi1p_D[c]
INFO:root:adding metadata to mi4p_D[c]
INFO:root:adding metadata to mi3p_D[c]
INFO:root:adding metadata to chitin[c]
INFO:root:adding metadata to acgam[c]
INFO:root:adding metadata to 23dhmb[m]
INFO:root:adding

INFO:root:adding metadata to dgtp[c]
INFO:root:adding metadata to dgmp[c]
INFO:root:adding metadata to dgdp[c]
INFO:root:adding metadata to dhor_S[c]
INFO:root:adding metadata to orot[c]
INFO:root:adding metadata to dhor_S[m]
INFO:root:adding metadata to orot[m]
INFO:root:adding metadata to spmd[c]
INFO:root:adding metadata to 4abutn[c]
INFO:root:adding metadata to 13dampp[c]
INFO:root:adding metadata to 2oxoadp[c]
INFO:root:adding metadata to glutcoa[c]
INFO:root:adding metadata to oxag[m]
INFO:root:adding metadata to 2oxoadp[m]
INFO:root:adding metadata to amuco[c]
INFO:root:adding metadata to L2aadp[c]
INFO:root:adding metadata to 4aphdob[c]
INFO:root:adding metadata to hLkynr[c]
INFO:root:adding metadata to dgsn[c]
INFO:root:adding metadata to id3acald[c]
INFO:root:adding metadata to 3hbcoa[x]
INFO:root:adding metadata to 3hbcoa[c]
INFO:root:adding metadata to hmgcoa[c]
INFO:root:adding metadata to 4aabutn[c]
INFO:root:adding metadata to frdp[c]
INFO:root:adding metadata to trdrd[c

INFO:root:adding metadata to pppg9[c]
INFO:root:adding metadata to pppg9[m]
INFO:root:adding metadata to 4mpetz[c]
INFO:root:adding metadata to 2mahmp[c]
INFO:root:adding metadata to 8aonn[c]
INFO:root:adding metadata to dann[c]
INFO:root:adding metadata to tag6p_D[c]
INFO:root:adding metadata to tagdp_D[c]
INFO:root:adding metadata to imacp[c]
INFO:root:adding metadata to 4ppcys[c]
INFO:root:adding metadata to lpam[m]
INFO:root:adding metadata to 4tmeabut[c]
INFO:root:adding metadata to 3htmelys[c]
INFO:root:adding metadata to 1p3h5c[c]
INFO:root:adding metadata to 4hpro_LT[m]
INFO:root:adding metadata to 1p3h5c[m]
INFO:root:adding metadata to 34hpl[m]
INFO:root:adding metadata to nicrns[m]
INFO:root:adding metadata to quln[c]
INFO:root:adding metadata to quln[m]
INFO:root:adding metadata to pi4p[c]
INFO:root:adding metadata to pi3p[c]
INFO:root:adding metadata to mi14p[c]
INFO:root:adding metadata to mi145p[c]
INFO:root:adding metadata to ptd2meeta_SC[c]
INFO:root:adding metadata to 

...

INFO:root:adding metadata to PSERDm_PP
INFO:root:adding metadata to ERGSTGLCT
INFO:root:adding metadata to EPISTESTH_PP
INFO:root:adding metadata to ERGSTESTH_PP
INFO:root:adding metadata to FECOSTESTH_PP
INFO:root:adding metadata to LANOSTESTH_PP
INFO:root:adding metadata to 2HMHMBQMTm
INFO:root:adding metadata to LALDO3
INFO:root:adding metadata to ADHAPR_PP
INFO:root:adding metadata to CYSTGS
INFO:root:adding metadata to PIN4K_PP
INFO:root:adding metadata to MFAPS_PP
INFO:root:adding metadata to PI4P5K_PP
INFO:root:adding metadata to 3MOPDC
INFO:root:adding metadata to SHCHD
INFO:root:adding metadata to 3DH5HPBMTm
INFO:root:adding metadata to 2HP6MPMOm
INFO:root:created term for R_2HP6MPMOm
INFO:root:adding metadata to 2HPMBQMTm
INFO:root:adding metadata to 2HPMMBQMOm
INFO:root:adding metadata to HBZOPT6m
INFO:root:adding metadata to 3OPHB5Hm
INFO:root:adding metadata to 3HPH5MBDCm
INFO:root:created term for R_3HPH5MBDCm
INFO:root:adding metadata to POLYAO3
INFO:root:adding metadata

INFO:root:adding metadata to Ex_adn
INFO:root:created term for R_Ex_adn
INFO:root:adding metadata to Ex_mal_L
INFO:root:created term for R_Ex_mal_L
INFO:root:adding metadata to Ex_akg
INFO:root:created term for R_Ex_akg
INFO:root:adding metadata to Ex_ala_D
INFO:root:created term for R_Ex_ala_D
INFO:root:adding metadata to Ex_ala_L
INFO:root:created term for R_Ex_ala_L
INFO:root:adding metadata to Ex_alltt
INFO:root:created term for R_Ex_alltt
INFO:root:adding metadata to Ex_amet
INFO:root:created term for R_Ex_amet
INFO:root:adding metadata to Ex_arab_D
INFO:root:created term for R_Ex_arab_D
INFO:root:adding metadata to Ex_arab_L
INFO:root:created term for R_Ex_arab_L
INFO:root:adding metadata to Ex_arg_L
INFO:root:created term for R_Ex_arg_L
INFO:root:adding metadata to Ex_btd_RR
INFO:root:created term for R_Ex_btd_RR
INFO:root:adding metadata to Ex_btn
INFO:root:created term for R_Ex_btn
INFO:root:adding metadata to Ex_chol
INFO:root:created term for R_Ex_chol
INFO:root:adding metad

INFO:root:adding metadata to FAS1_4
INFO:root:adding metadata to FAS1_6
INFO:root:adding metadata to FAS1_8
INFO:root:adding metadata to FAS1_10
INFO:root:adding metadata to FAS1_12
INFO:root:adding metadata to FAS1_14
INFO:root:adding metadata to FAS1_16
INFO:root:adding metadata to FAS1_18
INFO:root:adding metadata to FAS2_4
INFO:root:adding metadata to FAS2_6
INFO:root:adding metadata to FAS2_8
INFO:root:adding metadata to FAS2_10
INFO:root:adding metadata to FAS2_12
INFO:root:adding metadata to FAS2_14
INFO:root:adding metadata to FAS2_16
INFO:root:adding metadata to FAS2_18
INFO:root:adding metadata to FAS3_4
INFO:root:adding metadata to FAS3_6
INFO:root:adding metadata to FAS3_8
INFO:root:adding metadata to FAS3_10
INFO:root:adding metadata to FAS3_12
INFO:root:adding metadata to FAS3_14
INFO:root:adding metadata to FAS3_16
INFO:root:adding metadata to FAS3_18
INFO:root:adding metadata to FAS4_4
INFO:root:adding metadata to FAS4_6
INFO:root:adding metadata to FAS4_8
INFO:root:add

INFO:root:adding metadata to PALAt
INFO:root:created term for R_PALAt
INFO:root:adding metadata to APGSDpt
INFO:root:adding metadata to Ex_tur
INFO:root:created term for R_Ex_tur
INFO:root:adding metadata to TURt
INFO:root:created term for R_TURt
INFO:root:adding metadata to APGSDtn
INFO:root:adding metadata to HBAt
INFO:root:created term for R_HBAt
INFO:root:adding metadata to Ex_4hb
INFO:root:created term for R_Ex_4hb
INFO:root:adding metadata to usDHG
INFO:root:adding metadata to SSADH
INFO:root:adding metadata to Ex_drib
INFO:root:created term for R_Ex_drib
INFO:root:adding metadata to DRIBt
INFO:root:created term for R_DRIBt
INFO:root:adding metadata to ARABR2
INFO:root:adding metadata to Ex_mitril
INFO:root:created term for R_Ex_mitril
INFO:root:adding metadata to MITRILt
INFO:root:created term for R_MITRILt
INFO:root:adding metadata to PSIt
INFO:root:created term for R_PSIt
INFO:root:adding metadata to PSE
INFO:root:adding metadata to MsmEFGK
INFO:root:created term for R_MsmEFGK

1